In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import numpy as np

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

2023-04-19 21:07:15.001215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /Users/fengli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fengli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preprocessing


questions:
- what to do with stop words?

### Read data

In [4]:
with open('../data/pg1597.txt') as f:
    lines = f.readlines()
print(lines[:6])

["THE EMPEROR'S NEW CLOTHES\n", '\n', 'Many years ago, there was an Emperor, who was so excessively fond of\n', 'new clothes, that he spent all his money in dress. He did not trouble\n', 'himself in the least about his soldiers; nor did he care to go either to\n', 'the theatre or the chase, except for the opportunities then afforded him\n']


### Preprocess data

In [5]:
def preprocess(line):
    data = "".join(line)
    data = word_tokenize(data)
    words = " ".join(data)
    lower_w = words.lower()

    stop_words = set(stopwords.words('english')) 
    word_tokens = lower_w.split(" ")
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    ps = PorterStemmer()
    output = []
    for word in filtered_sentence:
        output.append((ps.stem(word)))
    final_out = " ".join(output)
    return final_out

### get vocabulary

In [6]:
def get_vocab(data):
    data = data.replace('\n', ' ').split(' ')
    words = list(set(data))
    vocabulary = {word:index for index, word in enumerate(words)}
    vocab_size = len(vocabulary)
    
    return vocabulary, vocab_size
    

In [7]:
train_data = preprocess(lines)
vocabulary, vocab_size = get_vocab(train_data)

In [8]:
vocab_size

4119

In [9]:
vocabulary

{'fell': 0,
 'queri': 1,
 'rub': 2,
 'silken': 3,
 'soon': 4,
 'mead': 5,
 'charm': 6,
 'restless': 7,
 'soldier': 8,
 'well-cram': 9,
 'sunni': 10,
 'sprang': 11,
 'string': 12,
 'others.': 13,
 'rosy-cheek': 14,
 'stamina': 15,
 'execution.': 16,
 'carriag': 17,
 'nothing.': 18,
 'circumst': 19,
 'desk': 20,
 'someon': 21,
 'nought': 22,
 'many-color': 23,
 'matter': 24,
 'gate': 25,
 'pursu': 26,
 'fork': 27,
 'ruin': 28,
 'citi': 29,
 'auction': 30,
 'burnt-out': 31,
 'hungri': 32,
 'palac': 33,
 'grapes.': 34,
 'viz': 35,
 'extent': 36,
 'vice': 37,
 'else.': 38,
 'honey-be': 39,
 'courteous': 40,
 'nichola': 41,
 'scarf': 42,
 'prescrib': 43,
 'ceil': 44,
 'pave': 45,
 'nest': 46,
 'bur': 47,
 'bell': 48,
 'again.': 49,
 'harm': 50,
 'pet': 51,
 'summer-tim': 52,
 'mistress': 53,
 'form': 54,
 'revel': 55,
 'bride': 56,
 'sink': 57,
 'perceiv': 58,
 '!': 59,
 'truli': 60,
 'christianshafen.': 61,
 'councillor': 62,
 'second': 63,
 'mr.': 64,
 'oak-tre': 65,
 'portrait': 66,
 'per

## Bi-directional LSTM Masked Language Modeling

references: 

https://keras.io/examples/nlp/masked_language_modeling/#create-bert-model-pretraining-model-for-masked-language-modeling

https://www.kaggle.com/code/ritvik1909/masked-language-modelling-rnn#Data-Preparation

https://keras.io/examples/nlp/bidirectional_lstm_imdb/

questions:
- should we split data by sentence instead of by fixed window size of 20?


### more data preparation

In [10]:
# add [mask] to vocabulary
mask_id = vocab_size
vocabulary['[mask]'] = mask_id

In [11]:
# convert words to vectors
vectorized_text = list(map(lambda x: vocabulary[x], train_data))
vectorized_text = np.array(y)

KeyError: 'e'

In [12]:
# split data into sequences of length 20
vectorized_text_len = len(vectorized_text) - (len(vectorized_text) % 20)
vectorized_text = vectorized_text[:vectorized_text_len]
vectorized_text = np.reshape(vectorized_text,[-1,20])

NameError: name 'vectorized_text' is not defined

In [13]:
vectorized_text

NameError: name 'vectorized_text' is not defined

In [171]:
def get_masked_input_label(sequence):
    
    # randomly choose one position in sequence to mask
    mask = np.random.randint(low=0, high=20)
    
    # add mask to input
    masked_input = [token if i != mask else mask_id for i, token in enumerate(sequence)]
    
    # set all values in label to -1(ignored by loss function) except the value at the masked position
    label = [-1 if i!= mask else token for i, token in enumerate(sequence)]
    return masked_input, label


In [152]:
# get masked inputs and labels
inputs = []
labels = []

for seq in vectorized_text:
    x,y = get_masked_input_label(seq)
    inputs.append(x)
    labels.append(y)
inputs = np.array(inputs)
labels = np.array(labels)
    

In [155]:
inputs.shape, labels.shape

((1820, 20), (1820, 20))

In [156]:
labels

array([[  -1,   -1,   -1, ...,   -1,   -1,   -1],
       [  -1,   -1,   -1, ...,   -1,   -1,   -1],
       [  -1,   -1,   -1, ...,   -1, 3330,   -1],
       ...,
       [  -1,   -1,   -1, ...,   -1,   -1,   -1],
       [  -1,   -1,   -1, ...,   -1,   -1,   -1],
       [  -1,   -1,   -1, ...,   -1, 1792,   -1]])

### bi-directional lstm model building and training

In [116]:
# define masked language modeling class
class LSTM_MLM(tf.keras.Model):
    def __init__(self, vocab_size, embed_size, input_length):
        """
        The Model class predicts the next words in a sequence.
        : param vocab_size : The number of unique words in the data
        : param hidden_size   : The size of your desired RNN
        : param embed_size : The size of your latent embedding
        """

        super().__init__()

        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.input_length = input_length

        ## TODO: define your trainable variables and/or layers here. This should include an
        ## embedding component, and any other variables/layers you require.

        # embedding layer
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size+1, output_dim=self.embed_size)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))
        self.dense1 = tf.keras.layers.Dense(self.vocab_size, activation='softmax')

        # fully connected linear layers


    def call(self, inputs):
        """
        You must use an embedding layer as the first layer of your network (i.e. tf.nn.embedding_lookup or tf.keras.layers.Embedding)
        :param inputs: word ids of shape (batch_size, 2)
        :return: logits: The batch element probabilities as a tensor of shape (batch_size, vocab_size)
        """

        # embedding layer
        x = inputs
        
        x = self.embedding(x)
        x = self.lstm(x)
        x = self.dense1(x)

        
        return x


In [159]:
model = LSTM_MLM(vocab_size, 64, 20)
loss_metric = tf.keras.losses.SparseCategoricalCrossentropy(ignore_class=-1)
model.compile(loss=loss_metric, optimizer='adam', metrics=['acc'])
model.fit(x=inputs, y=labels, validation_split=0.1, batch_size=20, epochs=50)

Epoch 1/50
82/82 [==============================] - 13s 86ms/step - loss: 7.1600 - acc: 0.0049 - val_loss: 6.8625 - val_acc: 0.0052
Epoch 2/50
82/82 [==============================] - 6s 72ms/step - loss: 5.6964 - acc: 0.0056 - val_loss: 7.1397 - val_acc: 0.0052
Epoch 3/50
82/82 [==============================] - 6s 77ms/step - loss: 5.3775 - acc: 0.0056 - val_loss: 7.3514 - val_acc: 0.0052
Epoch 4/50
82/82 [==============================] - 6s 75ms/step - loss: 5.1964 - acc: 0.0056 - val_loss: 8.0358 - val_acc: 0.0052
Epoch 5/50
82/82 [==============================] - 6s 70ms/step - loss: 5.0402 - acc: 0.0056 - val_loss: 8.0269 - val_acc: 0.0052
Epoch 6/50
82/82 [==============================] - 6s 70ms/step - loss: 4.8515 - acc: 0.0055 - val_loss: 8.3900 - val_acc: 0.0041
Epoch 7/50
82/82 [==============================] - 6s 70ms/step - loss: 4.6261 - acc: 0.0057 - val_loss: 9.1050 - val_acc: 0.0025
Epoch 8/50
82/82 [==============================] - 6s 70ms/step - loss: 4.3664 - 

In [172]:
# make prediction: still in progress

#query = ['emperor', 'like', '[mask]', 'cloth', 'dress']
#query_id = [vocabulary[q] for q in query]
query_id = x[0:2]

pred = model(query_id)
pred

AttributeError: Exception encountered when calling layer 'embedding_13' (type Embedding).

'list' object has no attribute 'dtype'

Call arguments received by layer 'embedding_13' (type Embedding):
  • inputs=['3577', '3205']

## Transformers

references: "Attention Is All You Need" paper by Vaswani et al.

In [ ]:
# Multi-head self-attention
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        assert self.embed_dim % self.num_heads == 0, "Embedding dimension must be divisible by the number of heads."

        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

In [ ]:
# Transformer block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"),
             tf.keras.layers.Dense(embed_dim)]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
# Transformer model
class Transformer(tf.keras.Model):
    def __init__(self, vocab_size, input_length, embed_dim, num_heads, ff_dim, num_blocks, rate=0.1):
        super(Transformer, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size + 1, embed_dim)
        self.pos_encoding = self.positional_encoding(input_length, embed_dim)
        self.transformer_blocks = [TransformerBlock(embed_dim, num_heads, ff_dim, rate) for _ in range(num_blocks)]
        self.dropout = tf.keras.layers.Dropout(rate)
        self.dense = tf.keras.layers.Dense(vocab_size, activation="softmax")

    def positional_encoding(self, input_length, embed_dim):
        angles = 1 / (10000 ** (tf.range(0, embed_dim, 2, dtype=tf.float32) / embed_dim))
        pos_encodings = tf.range(input_length, dtype=tf.float32)[:, tf.newaxis] * angles
        pos_encodings = tf.concat([tf.sin(pos_encodings), tf.cos(pos_encodings)], axis=-1)
        pos_encodings = pos_encodings[tf.newaxis, ...]
        return tf.cast(pos_encodings, tf.float32)

    def call(self, inputs, training):
        seq_length = inputs.shape[1]
        embeddings = self.embedding(inputs)
        embeddings *= tf.math.sqrt(tf.cast(self.embedding.output_dim, tf.float32))
        embeddings += self.pos_encoding[:, :seq_length, :]

        x = self.dropout(embeddings, training=training)

        for transformer_block in self.transformer_blocks:
            x = transformer_block(x, training)

        logits = self.dense(x)
        return logits

In [ ]:
# Hyperparameters
vocab_size = 10000
input_length = 20
embed_dim = 64
num_heads = 4
ff_dim = 128
num_blocks = 4

# Instantiate the model
model_t = Transformer(vocab_size, input_length, embed_dim, num_heads, ff_dim, num_blocks)

# Set up the loss function and optimizer
loss_metric = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, ignore_index=-1)
optimizer = tf.keras.optimizers.Adam()

model_t.compile(loss=loss_metric, optimizer=optimizer, metrics=['accuracy'])

# Train the model
model_t.fit(x=inputs, y=labels, validation_split=0.1, batch_size=20, epochs=50)